In [19]:
import os
from dotenv import load_dotenv
import asyncio
from autogen_agentchat.ui import Console
from autogen_core.models import  SystemMessage, UserMessage
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
import openai 
from openai import AsyncAzureOpenAI
from typing import List
import json
import string
import uuid


import asyncio
from dataclasses import dataclass

from autogen_core import (
    DefaultTopicId,
    FunctionCall,
    Image,
    MessageContext,
    RoutedAgent,
    SingleThreadedAgentRuntime,
    TopicId,
    TypeSubscription,
    message_handler,
)
from autogen_core.models import (
    AssistantMessage,
    LLMMessage,
    SystemMessage,
    UserMessage,
)
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient
from IPython.display import display  # type: ignore
from pydantic import BaseModel
from rich.console import Console
from rich.markdown import Markdown

# Load environment variables from a .env file
load_dotenv("/home/azureuser/azure-ai-agent-workshop/.env")

True

In [20]:
model_client=AzureOpenAIChatCompletionClient(
            model=os.getenv("AZURE_OPENAI_MODEL"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_KEY"),
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
            api_version=os.getenv("AZUER_OPENAI_API_VERSION")
        )

In [21]:

dall_e_client = AsyncAzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_DALL_E_ENDPOINT"),
    api_version=os.getenv("AZUER_OPENAI_DALL_E_API_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_DALL_E_DEPLOYMENT"),
    api_key=os.getenv("AZURE_OPENAI_DALL_E_KEY")
)


In [22]:
class GroupChatMessage(BaseModel):
    body: UserMessage


class RequestToSpeak(BaseModel):
    pass

In [23]:
class BaseGroupChatAgent(RoutedAgent):
    """A group chat participant using an LLM."""

    def __init__(
        self,
        description: str,
        group_chat_topic_type: str,
        model_client: OpenAIChatCompletionClient,
        system_message: str,
    ) -> None:
        super().__init__(description=description)
        self._group_chat_topic_type = group_chat_topic_type
        self._model_client = model_client
        self._system_message = SystemMessage(content=system_message)
        self._chat_history: List[LLMMessage] = []

    @message_handler
    async def handle_message(self, message: GroupChatMessage, ctx: MessageContext) -> None:
        self._chat_history.extend(
            [
                UserMessage(content=f"Transferred to {message.body.source}", source="system"),
                message.body,
            ]
        )

    @message_handler
    async def handle_request_to_speak(self, message: RequestToSpeak, ctx: MessageContext) -> None:
        # print(f"\n{'-'*80}\n{self.id.type}:", flush=True)
        Console().print(Markdown(f"### {self.id.type}: "))
        self._chat_history.append(
            UserMessage(content=f"Transferred to {self.id.type}, adopt the persona immediately.", source="system")
        )
        completion = await self._model_client.create([self._system_message] + self._chat_history)
        assert isinstance(completion.content, str)
        self._chat_history.append(AssistantMessage(content=completion.content, source=self.id.type))
        Console().print(Markdown(completion.content))
        # print(completion.content, flush=True)
        await self.publish_message(
            GroupChatMessage(body=UserMessage(content=completion.content, source=self.id.type)),
            topic_id=DefaultTopicId(type=self._group_chat_topic_type),
        )


In [24]:
class WriterAgent(BaseGroupChatAgent):
    def __init__(self, description: str, group_chat_topic_type: str, model_client: OpenAIChatCompletionClient) -> None:
        super().__init__(
            description=description,
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
            system_message="You are a Writer. You produce good work.",
        )


class EditorAgent(BaseGroupChatAgent):
    def __init__(self, description: str, group_chat_topic_type: str, model_client: OpenAIChatCompletionClient) -> None:
        super().__init__(
            description=description,
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
            system_message="You are an Editor. Plan and guide the task given by the user. Provide critical feedbacks to the draft and illustration produced by Writer and Illustrator. "
            "Approve if the task is completed and the draft and illustration meets user's requirements.",
        )


In [25]:
class IllustratorAgent(BaseGroupChatAgent):
    def __init__(
        self,
        description: str,
        group_chat_topic_type: str,
        model_client: OpenAIChatCompletionClient,
        image_client: openai.AsyncClient,
    ) -> None:
        super().__init__(
            description=description,
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
            system_message="You are an Illustrator. You use the generate_image tool to create images given user's requirement. "
            "Make sure the images have consistent characters and style.",
        )
        self._image_client = image_client
        self._image_gen_tool = FunctionTool(
            self._image_gen, name="generate_image", description="Call this to generate an image. "
        )

    async def _image_gen(
        self, character_appearence: str, style_attributes: str, worn_and_carried: str, scenario: str
    ) -> str:
        prompt = f"Digital painting of a {character_appearence} character with {style_attributes}. Wearing {worn_and_carried}, {scenario}."
        response = await self._image_client.images.generate(
            prompt=prompt, model="dall-e-3", response_format="b64_json", size="1024x1024"
        )
        return response.data[0].b64_json  # type: ignore

    @message_handler
    async def handle_request_to_speak(self, message: RequestToSpeak, ctx: MessageContext) -> None:  # type: ignore
        Console().print(Markdown(f"### {self.id.type}: "))
        self._chat_history.append(
            UserMessage(content=f"Transferred to {self.id.type}, adopt the persona immediately.", source="system")
        )
        # Ensure that the image generation tool is used.
        completion = await self._model_client.create(
            [self._system_message] + self._chat_history,
            tools=[self._image_gen_tool],
            extra_create_args={"tool_choice": "required"},
            cancellation_token=ctx.cancellation_token,
        )
        assert isinstance(completion.content, list) and all(
            isinstance(item, FunctionCall) for item in completion.content
        )
        images: List[str | Image] = []
        for tool_call in completion.content:
            arguments = json.loads(tool_call.arguments)
            Console().print(arguments)
            result = await self._image_gen_tool.run_json(arguments, ctx.cancellation_token)
            image = Image.from_base64(self._image_gen_tool.return_value_as_string(result))
            image = Image.from_pil(image.image.resize((256, 256)))
            display(image.image)  # type: ignore
            images.append(image)
        await self.publish_message(
            GroupChatMessage(body=UserMessage(content=images, source=self.id.type)),
            DefaultTopicId(type=self._group_chat_topic_type),
        )

In [26]:
class UserAgent(RoutedAgent):
    def __init__(self, description: str, group_chat_topic_type: str) -> None:
        super().__init__(description=description)
        self._group_chat_topic_type = group_chat_topic_type

    @message_handler
    async def handle_message(self, message: GroupChatMessage, ctx: MessageContext) -> None:
        # When integrating with a frontend, this is where group chat message would be sent to the frontend.
        pass

    @message_handler
    async def handle_request_to_speak(self, message: RequestToSpeak, ctx: MessageContext) -> None:
        user_input = input("Enter your message, type 'APPROVE' to conclude the task: ")
        Console().print(Markdown(f"### User: \n{user_input}"))
        await self.publish_message(
            GroupChatMessage(body=UserMessage(content=user_input, source=self.id.type)),
            DefaultTopicId(type=self._group_chat_topic_type),
        )


In [27]:
class GroupChatManager(RoutedAgent):
    def __init__(
        self,
        participant_topic_types: List[str],
        model_client: OpenAIChatCompletionClient,
        participant_descriptions: List[str],
    ) -> None:
        super().__init__("Group chat manager")
        self._participant_topic_types = participant_topic_types
        self._model_client = model_client
        self._chat_history: List[UserMessage] = []
        self._participant_descriptions = participant_descriptions
        self._previous_participant_topic_type: str | None = None

    @message_handler
    async def handle_message(self, message: GroupChatMessage, ctx: MessageContext) -> None:
        assert isinstance(message.body, UserMessage)
        self._chat_history.append(message.body)
        # If the message is an approval message from the user, stop the chat.
        if message.body.source == "User":
            assert isinstance(message.body.content, str)
            if message.body.content.lower().strip(string.punctuation).endswith("approve"):
                return
        # Format message history.
        messages: List[str] = []
        for msg in self._chat_history:
            if isinstance(msg.content, str):
                messages.append(f"{msg.source}: {msg.content}")
            elif isinstance(msg.content, list):
                line: List[str] = []
                for item in msg.content:
                    if isinstance(item, str):
                        line.append(item)
                    else:
                        line.append("[Image]")
                messages.append(f"{msg.source}: {', '.join(line)}")
        history = "\n".join(messages)
        # Format roles.
        roles = "\n".join(
            [
                f"{topic_type}: {description}".strip()
                for topic_type, description in zip(
                    self._participant_topic_types, self._participant_descriptions, strict=True
                )
                if topic_type != self._previous_participant_topic_type
            ]
        )
        selector_prompt = """You are in a role play game. The following roles are available:
{roles}.
Read the following conversation. Then select the next role from {participants} to play. Only return the role.

{history}

Read the above conversation. Then select the next role from {participants} to play. Only return the role.
"""
        system_message = SystemMessage(
            content=selector_prompt.format(
                roles=roles,
                history=history,
                participants=str(
                    [
                        topic_type
                        for topic_type in self._participant_topic_types
                        if topic_type != self._previous_participant_topic_type
                    ]
                ),
            )
        )
        completion = await self._model_client.create([system_message], cancellation_token=ctx.cancellation_token)
        assert isinstance(completion.content, str)
        selected_topic_type: str
        for topic_type in self._participant_topic_types:
            if topic_type.lower() in completion.content.lower():
                selected_topic_type = topic_type
                self._previous_participant_topic_type = selected_topic_type
                await self.publish_message(RequestToSpeak(), DefaultTopicId(type=selected_topic_type))
                return
        raise ValueError(f"Invalid role selected: {completion.content}")

In [29]:
runtime = SingleThreadedAgentRuntime()

editor_topic_type = "Editor"
writer_topic_type = "Writer"
illustrator_topic_type = "Illustrator"
user_topic_type = "User"
group_chat_topic_type = "group_chat"

editor_description = "Editor for planning and reviewing the content."
writer_description = "Writer for creating any text content."
user_description = "User for providing final approval."
illustrator_description = "An illustrator for creating images."

editor_agent_type = await EditorAgent.register(
    runtime,
    editor_topic_type,  # Using topic type as the agent type.
    lambda: EditorAgent(
        description=editor_description,
        group_chat_topic_type=group_chat_topic_type,
        model_client=model_client,
    ),
)
await runtime.add_subscription(TypeSubscription(topic_type=editor_topic_type, agent_type=editor_agent_type.type))
await runtime.add_subscription(TypeSubscription(topic_type=group_chat_topic_type, agent_type=editor_agent_type.type))

writer_agent_type = await WriterAgent.register(
    runtime,
    writer_topic_type,  # Using topic type as the agent type.
    lambda: WriterAgent(
        description=writer_description,
        group_chat_topic_type=group_chat_topic_type,
        model_client=model_client,
    ),
)
await runtime.add_subscription(TypeSubscription(topic_type=writer_topic_type, agent_type=writer_agent_type.type))
await runtime.add_subscription(TypeSubscription(topic_type=group_chat_topic_type, agent_type=writer_agent_type.type))

illustrator_agent_type = await IllustratorAgent.register(
    runtime,
    illustrator_topic_type,
    lambda: IllustratorAgent(
        description=illustrator_description,
        group_chat_topic_type=group_chat_topic_type,
        model_client=model_client,
        image_client=dall_e_client,
    ),
)
await runtime.add_subscription(
    TypeSubscription(topic_type=illustrator_topic_type, agent_type=illustrator_agent_type.type)
)
await runtime.add_subscription(
    TypeSubscription(topic_type=group_chat_topic_type, agent_type=illustrator_agent_type.type)
)

user_agent_type = await UserAgent.register(
    runtime,
    user_topic_type,
    lambda: UserAgent(description=user_description, group_chat_topic_type=group_chat_topic_type),
)
await runtime.add_subscription(TypeSubscription(topic_type=user_topic_type, agent_type=user_agent_type.type))
await runtime.add_subscription(TypeSubscription(topic_type=group_chat_topic_type, agent_type=user_agent_type.type))

group_chat_manager_type = await GroupChatManager.register(
    runtime,
    "group_chat_manager",
    lambda: GroupChatManager(
        participant_topic_types=[writer_topic_type, illustrator_topic_type, editor_topic_type, user_topic_type],
        model_client=model_client,
        participant_descriptions=[writer_description, illustrator_description, editor_description, user_description],
    ),
)
await runtime.add_subscription(
    TypeSubscription(topic_type=group_chat_topic_type, agent_type=group_chat_manager_type.type)
)

In [30]:
runtime.start()
session_id = str(uuid.uuid4())
await runtime.publish_message(
    GroupChatMessage(
        body=UserMessage(
            content="Please write a short story about the gingerbread man with up to 3 photo-realistic illustrations.",
            source="User",
        )
    ),
    TopicId(type=group_chat_topic_type, source=session_id),
)
await runtime.stop_when_idle()

Writer:

The Gingerbread Man's Great Escape                                                                                 

On a frosty morning in a cozy little village, the sun peeked over the snow-capped rooftops, casting a warm glow on 
Mrs. Baker’s gingerbread shop. The delightful aroma of spices and sugar wafted through the air as she prepared her 
special batch of gingerbread men.                                                                                  

Illustration 1: A quaint gingerbread shop nestled in a snowy village, with smoke curling from its chimney and      
colorful decorations in the window. Inside, Mrs. Baker, a kindly older woman with rosy cheeks, rolls out dough     
while gingerbread cookies, cut into the shapes of little men and women, await baking.                              

Once baked, her gingerbread men came alive with piped icing eyes and buttons of candy. But one gingerbread man was 
different; he was infused with a sprinkle of magic. As soon as he was placed on the cooling rack, he sprang to     
life, laughing with a sprightly voice. “Run, run, as fast as you can! You can’t catch me, I’m the gingerbread man!”

Mrs. Baker could hardly believe her eyes as he dashed past her, out the door, and onto the snowy streets. With     
curiosity ignited and a twinkle of adventure in his sugary eyes, the gingerbread man sprinted across the village   
square, leaving a trail of sweet crumbs behind him.                                                                

Illustration 2: The gingerbread man running joyfully through the village square, his icing eyes gleaming and candy 
buttons shining. Villagers gaze out of their windows in awe, while children chase after him, laughter echoing in   
the crisp air.                                                                                                     

As panicked villagers and children chased after him, the gingerbread man shouted, “You can’t catch me!” He raced   
across the fields, through patches of frosty grass and over a babbling brook. Just behind him, an old farmer, his  
face flushed with bewilderment, yelled, “Stop! I want to eat you!”                                                 

But the gingerbread man zigzagged through the trees, his frosted grin never fading. “You can’t catch me! I’m the   
gingerbread man!”                                                                                                  

He soon encountered a clever fox lounging beside the water’s edge. The fox, intrigued by the sight of the          
delightful treat, licked his lips. “I won’t chase you like the others,” he said smoothly. “I can help you get      
across the river. Why not hop on my back?”                                                                         

Illustration 3: The gingerbread man standing at the edge of a shimmering river, looking at the sly fox with a mix  
of curiosity and caution. The fox smiles, his eyes glinting with mischief, while the sun casts a golden light over 
the scene.                                                                                                         

For a moment, the gingerbread man hesitated. But with a dash of bravery, he leapt onto the fox’s back. The fox     
grinned, a cunning sparkle in his eyes as he began to wade into the water.                                         

Halfway across, the fox announced, “You look heavy. Why don’t you sit on my nose instead? It will be easier for me,
and you’ll have a better view!” Trusting the fox’s words, the gingerbread man hopped onto his nose.                

In an instant, the sly fox tossed him high into the air and opened his mouth wide. “You’re finally mine!” he       
exclaimed, but the gingerbread man, being agile and quick, flipped in mid-air and landed safely on the riverbank.  

With a final triumphant laugh, he called out, “You can’t catch me! I’m the gingerbread man!” With that, he dashed  
into the sunset, his spirit unbroken, forev

Editor:

Thank you for sharing your short story about the gingerbread man along with the proposed illustrations. Here are my
thoughts and feedback on each element:                                                                             

                                                  Story Feedback:                                                  

 1 Concept and Structure: The narrative flows well and captures the classic charm of the gingerbread man tale. The 
   incremental build-up of excitement from the baking to the chase is engaging.                                    
 2 Characterization: The gingerbread man is portrayed vividly with his adventurous spirit, which is great. However,
   consider developing Mrs. Baker's character a bit more—perhaps by including her reaction to the gingerbread man's
   antics. This could add depth to the story.                                                                      
 3 Pacing: The pacing is generally good, but there is a slight rush towards the ending. It might benefit from a bit
   more tension or conflict with the fox before the resolution. The fox's plot twist is clever, but the gingerbread
   man’s quick escape seems a bit abrupt.                                                                          
 4 Language: The language is playful and appropriate for a children’s story. Consider incorporating more sensory   
   descriptions—smells and sounds—that could enhance the vividness of the scenes.                                  

                                              Illustration Feedback:                                               

 1 Illustration 1: This scene nicely establishes the setting. Ensure the warmth of Mrs. Baker’s character shines   
   through her expression and the cozy details of the shop. Adding a bit more color and warmth could enhance the   
   inviting atmosphere.                                                                                            
 2 Illustration 2: The depiction of the gingerbread man in motion is delightful. However, adding more dynamic      
   elements—like fluttering snowflakes or excited expressions from the villagers—could enhance the sense of        
   movement and joy.                                                                                               
 3 Illustration 3: The contrast between the gleaming river and the sly fox is effective. To enhance this scene     
   further, consider depicting more tension, maybe showing the gingerbread man looking hesitant or the fox with a  
   more pronounced cunning expression.                                                                             

                                               Overall Impressions:                                                

The story is well-conceived and full of charm, but it would benefit from minor adjustments to character depth and  
pacing. The illustrations are promising but could use a bit more dynamic elements to fully capture the excitement  
of the chase.                                                                                                      

Once these adjustments are made, I believe the story and illustrations will create a delightful experience for     
readers. Please proceed with revising based on this feedback, and I look forward to seeing the updated version!

Writer:

Thank you for your thoughtful feedback! I appreciate your insights and will work on revising the story to          
incorporate your suggestions. Here’s an updated version that addresses the points you raised:                      

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
The Gingerbread Man's Great Escape                                                                                 

On a frosty morning in the cozy village of Sugar Hollow, the sun peeked over the snow-capped rooftops, casting a   
warm glow on Mrs. Baker’s charming gingerbread shop. The delightful aroma of spices and sugar wafted through the   
air as she mixed her special batter for the gingerbread men.                                                       

Illustration 1: A quaint gingerbread shop nestled in a snowy village, with smoke curling from its chimney. Inside, 
Mrs. Baker—a cheerful woman with rosy cheeks and a flour-dusted apron—smiles as she rolls out dough, and charming  
gingerbread cookies, cut into shapes of little men and women, fill the baker's counter. The shop is filled with    
colorful decorations that twinkle like holiday lights.                                                             

Once baked, her gingerbread men emerged from the oven, adorned with piped icing eyes and buttons made of rainbow   
candy. But one gingerbread man was different; he was infused with a sprinkle of magic. As soon as he was placed on 
the cooling rack, he sprang to life with a hearty laugh. “Run, run, as fast as you can! You can’t catch me, I’m the
gingerbread man!”                                                                                                  

Mrs. Baker gasped, her eyes wide with surprise and delight. “Oh my goodness! Come back!” she called, but the       
gingerbread man had already dashed out the door and onto the snowy streets. With curiosity ignited and a twinkle of
adventure in his sugary eyes, he sprinted across the village square, leaving a trail of sweet crumbs behind him.   

Illustration 2: The gingerbread man running joyfully through the bustling village square, his icing eyes glimmering
and candy buttons sparkling. Villagers gaze out of their windows in awe while children chase after him, laughter   
echoing in the crisp air. Snowflakes flutter down, adding to the festive atmosphere.                               

As panicked villagers and children chased after him, the gingerbread man shouted, “You can’t catch me!” He dashed  
across the fields, soaring over patches of frosty grass and finally bounding over a babbling brook. Just behind    
him, a startled farmer, his face flushed with astonishment, yelled, “Stop! I want to eat you!”                     

But the gingerbread man zigzagged through the trees, his frosted smile unwavering. “You can’t catch me! I'm the    
gingerbread man!”                                                                                                  

He soon encountered a cunning fox lounging beside the water’s edge. The fox, intrigued by the sight of the         
delightful treat, licked his lips and grinned slyly. “I won’t chase you like the others,” he said, his voice smooth
as syrup. “I can help you get across the river. Just hop on my back!”                                              

For a moment, the gingerbread man hesitated, the rush of the river bubbling behind him. But with a dash of bravery 
and the spirit of adventure calling to him, he leapt onto the fox’s back, trusting his words.                      

Illustration 3: The gingerbread man perched precariously on the sly fox's back at the shimmering river’s edge,     
looking both excited and cautious. The fox smiles with mischief, the sunlight reflecting off the water, while the  
gingerbread man’s expression hints at his inner conflict.                                                          

Halfway across, the fox began to wobble. “Yo

User:                                                       

approve